# Feed Forward Neural Networks

In [ ]:
# jupyter notebook

In [ ]:
### 210701~210711, 210821~210831 검증기간(총 22일간)

In [24]:
from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
from keras import Input
from keras.layers import Dense

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [2]:
from keras.callbacks import EarlyStopping

# Train

In [3]:
cd C:\weather\v3\data\train

C:\weather\v3\data\train


In [4]:
forders_tr = os.listdir()
forders_tr

['table_202001_v3.csv',
 'table_202002_v3.csv',
 'table_202003_v3.csv',
 'table_202004_v3.csv',
 'table_202005_v3.csv',
 'table_202006_v3.csv',
 'table_202007_v3.csv',
 'table_202008_v3.csv',
 'table_202009_v3.csv',
 'table_202010_v3.csv',
 'table_202011_v3.csv',
 'table_202012_v3.csv',
 'table_202101_v3.csv',
 'table_202102_v3.csv',
 'table_202103_v3.csv',
 'table_202104_v3.csv',
 'table_202105_v3.csv',
 'table_202106_v3.csv']

In [5]:
# 새로운 데이터 프레임 생성
train = pd.DataFrame()

for files in forders_tr:
    df= pd.read_csv(files)
    train = pd.concat([train, df])

In [6]:
train

,YearMonthDayHourMinute,STN,Lon,Lat,isitu-LST,Band1,Band2,Band3,Band4,Band5,...,Year,Month,Day,Hour,Minute,mean_6_23,mean_34_37,mean_38_39,mean_40_42,mean_43_44
0,202001010000,90,128.56473,38.25085,-2.6,0.00000,0.00000,-0.00060,-0.00030,-0.0003,...,2020,1,1,0,0,0.422121,0.000000,0.000000,0.000000,0.868476
1,202001010000,95,127.30420,38.14787,-2.6,-0.00057,-0.00057,0.00030,-0.00060,-0.0003,...,2020,1,1,0,0,-0.391234,0.000000,0.000000,0.000000,1.114533
2,202001010000,98,127.06070,37.90188,-3.1,0.00000,0.00000,0.00030,0.00060,-0.0003,...,2020,1,1,0,0,-0.355405,0.000000,0.000000,0.000000,1.315872
3,202001010000,99,126.76648,37.88589,-3.4,-0.00057,-0.00057,-0.00030,0.00015,0.0000,...,2020,1,1,0,0,-0.332510,0.000000,0.000000,0.000000,1.666385
4,202001010000,101,127.73570,37.90262,-3.6,-0.00057,0.00000,0.00030,0.00030,0.0000,...,2020,1,1,0,0,-0.188191,-1.022276,-0.499305,0.114355,1.402172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447705,202106302350,973,126.94961,35.02330,23.6,-0.00057,-0.00057,-0.00179,-0.00060,-0.0003,...,2021,6,30,23,50,0.000385,1.107682,1.107273,0.000000,0.209894
447706,202106302350,974,128.15605,35.24435,20.7,0.00000,-0.00057,0.00089,0.00015,0.0000,...,2021,6,30,23,50,0.071010,1.472435,1.488681,0.000000,0.000000
447707,202106302350,977,127.46688,36.72455,23.9,-0.00057,-0.00057,-0.00060,0.00015,0.0000,...,2021,6,30,23,50,-0.144802,1.049670,1.210674,0.000000,0.000000
447708,202106302350,978,127.77626,37.95461,23.2,-0.00057,0.00000,0.00030,-0.00075,-0.0003,...,2021,6,30,23,50,-0.365471,0.726709,0.740235,0.000000,0.000000


# Validation

In [7]:
cd C:\weather\v3\data\val

C:\weather\v3\data\val


In [8]:
forders_val = os.listdir()
forders_val

['table_202109_v3.csv',
 'table_202110_v3.csv',
 'table_202111_v3.csv',
 'table_202112_v3.csv']

In [9]:
# 새로운 데이터 프레임 생성
val = pd.DataFrame()

for files in forders_val:
    df= pd.read_csv(files)
    val = pd.concat([val, df])

In [10]:
val

,YearMonthDayHourMinute,STN,Lon,Lat,isitu-LST,Band1,Band2,Band3,Band4,Band5,...,Year,Month,Day,Hour,Minute,mean_6_23,mean_34_37,mean_38_39,mean_40_42,mean_43_44
0,202109010000,90,128.56473,38.25085,17.9,0.00170,0.00171,0.00238,0.00195,0.00181,...,2021,9,1,0,0,-0.733944,0.000000,0.000000,0.000000,0.508451
1,202109010000,93,127.75443,37.94738,19.0,0.00170,0.00171,0.00149,0.00180,0.00121,...,2021,9,1,0,0,-0.696051,-0.914975,0.447435,0.341129,-0.179931
2,202109010000,95,127.30420,38.14787,17.6,0.00170,0.00228,0.00179,0.00195,0.00151,...,2021,9,1,0,0,-0.279810,0.000000,0.000000,0.000000,-0.365214
3,202109010000,98,127.06070,37.90188,19.3,0.00227,0.00228,0.00238,0.00374,0.00181,...,2021,9,1,0,0,-0.195837,0.000000,0.000000,0.000000,-0.290579
4,202109010000,99,126.76648,37.88589,19.6,0.00227,0.00228,0.00268,0.00239,0.00151,...,2021,9,1,0,0,-0.370219,0.000000,0.000000,0.000000,0.015434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432625,202112312350,702,126.99435,35.94504,-3.2,-0.00057,-0.00057,-0.00030,-0.00045,-0.00030,...,2021,12,31,23,50,-0.787369,-1.417396,-1.534663,0.000000,0.000000
432626,202112312350,972,128.68340,36.54983,-4.8,-0.00057,0.00057,0.00089,-0.00015,0.00000,...,2021,12,31,23,50,-0.672741,-1.492066,-1.894582,0.000000,1.185870
432627,202112312350,973,126.94961,35.02330,-2.4,0.00000,-0.00057,-0.00179,0.00015,-0.00030,...,2021,12,31,23,50,-0.684352,-0.847900,-0.716603,0.000000,1.525860
432628,202112312350,974,128.15605,35.24435,-1.8,-0.00057,-0.00057,0.00060,-0.00015,0.00000,...,2021,12,31,23,50,-2.065335,-1.364155,-0.756098,0.000000,0.000000


# Test

In [11]:
cd C:\weather\v3\data\test

C:\weather\v3\data\test


In [12]:
forders_te = os.listdir()
forders_te

['table_202107_1_v3.csv',
 'table_202108_2_v3.csv',
 'zNex~$haretable_202107_1_v3.csv']

In [13]:
### 210701~210711, 210821~210831 검증기간(총 22일간)

# 새로운 데이터 프레임 생성
test = pd.DataFrame()

for files in forders_te:
    df= pd.read_csv(files)
    test = pd.concat([test, df])

In [14]:
test

,YearMonthDayHourMinute,STN,Lon,Lat,Band1,Band2,Band3,Band4,Band5,Band6,...,Month,Day,Hour,Minute,mean_6_23,mean_34_37,mean_38_39,mean_40_42,mean_43_44,H
0,2.021070e+11,90.0,128.56473,38.25085,-0.00057,-0.00057,-0.00149,-0.00045,-0.00060,0.00000,...,7.0,0.0,0.0,0.0,-0.137869,0.000000,0.000000,0.000000,0.503333,NaN
1,2.021070e+11,93.0,127.75443,37.94738,-0.00057,-0.00057,-0.00119,0.00000,0.00000,-0.00065,...,7.0,0.0,0.0,0.0,0.115111,-0.141101,0.027567,-0.354792,0.074239,NaN
2,2.021070e+11,95.0,127.30420,38.14787,-0.00057,-0.00057,-0.00060,0.00015,0.00000,-0.00065,...,7.0,0.0,0.0,0.0,-0.072823,0.000000,0.000000,0.000000,-0.073873,NaN
3,2.021070e+11,98.0,127.06070,37.90188,-0.00057,-0.00057,0.00030,-0.00015,-0.00030,-0.00065,...,7.0,0.0,0.0,0.0,0.138902,0.000000,0.000000,0.000000,0.065445,NaN
4,2.021070e+11,99.0,126.76648,37.88589,-0.00057,0.00000,-0.00060,-0.00105,0.00000,-0.00065,...,7.0,0.0,0.0,0.0,0.177468,0.000000,0.000000,0.000000,0.444882,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27087,2.021080e+11,973.0,126.94961,35.02330,0.00000,0.00000,0.00030,0.00015,0.00121,0.00000,...,8.0,0.0,0.0,0.0,-0.207862,1.366354,0.637043,0.000000,0.664962,NaN
27088,2.021080e+11,974.0,128.15605,35.24435,0.00000,0.00057,0.00000,0.00015,0.00000,0.00000,...,8.0,0.0,0.0,0.0,-0.559507,1.766315,1.253310,0.000000,0.000000,NaN
27089,2.021080e+11,977.0,127.46688,36.72455,0.00000,0.00057,-0.00179,0.00030,0.00000,0.00000,...,8.0,0.0,0.0,0.0,-0.449989,-0.705848,-0.467874,0.000000,0.000000,NaN
27090,2.021080e+11,978.0,127.77626,37.95461,0.00000,0.00000,0.00089,0.00015,-0.00030,0.00000,...,8.0,0.0,0.0,0.0,-0.374500,-1.392848,-0.768451,0.000000,0.000000,NaN


# Submission

In [15]:
cd C:\weather\data

C:\weather\data


In [16]:
# Set Pandas options to display more columns
# pd.options.display.max_columns=50
submission=pd.read_csv('1-2_검증데이터셋.csv', encoding='utf-8')
submission

,YearMonthDayHourMinute,STN,isitu-LST,insitu-TA
0,202107010030,90,NaN,NaN
1,202107010030,93,NaN,NaN
2,202107010030,95,NaN,NaN
3,202107010030,98,NaN,NaN
4,202107010030,99,NaN,NaN
...,...,...,...,...
54498,202108312330,973,NaN,NaN
54499,202108312330,974,NaN,NaN
54500,202108312330,977,NaN,NaN
54501,202108312330,978,NaN,NaN


# 모델 Train

In [25]:
train

,YearMonthDayHourMinute,STN,Lon,Lat,isitu-LST,Band1,Band2,Band3,Band4,Band5,...,Year,Month,Day,Hour,Minute,mean_6_23,mean_34_37,mean_38_39,mean_40_42,mean_43_44
0,202001010000,90,128.56473,38.25085,-2.6,0.00000,0.00000,-0.00060,-0.00030,-0.0003,...,2020,1,1,0,0,0.422121,0.000000,0.000000,0.000000,0.868476
1,202001010000,95,127.30420,38.14787,-2.6,-0.00057,-0.00057,0.00030,-0.00060,-0.0003,...,2020,1,1,0,0,-0.391234,0.000000,0.000000,0.000000,1.114533
2,202001010000,98,127.06070,37.90188,-3.1,0.00000,0.00000,0.00030,0.00060,-0.0003,...,2020,1,1,0,0,-0.355405,0.000000,0.000000,0.000000,1.315872
3,202001010000,99,126.76648,37.88589,-3.4,-0.00057,-0.00057,-0.00030,0.00015,0.0000,...,2020,1,1,0,0,-0.332510,0.000000,0.000000,0.000000,1.666385
4,202001010000,101,127.73570,37.90262,-3.6,-0.00057,0.00000,0.00030,0.00030,0.0000,...,2020,1,1,0,0,-0.188191,-1.022277,-0.499305,0.114355,1.402172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447705,202106302350,973,126.94961,35.02330,23.6,-0.00057,-0.00057,-0.00179,-0.00060,-0.0003,...,2021,6,30,23,50,0.000385,1.107697,1.107273,0.000000,0.209894
447706,202106302350,974,128.15605,35.24435,20.7,0.00000,-0.00057,0.00089,0.00015,0.0000,...,2021,6,30,23,50,0.071010,1.472452,1.488681,0.000000,0.000000
447707,202106302350,977,127.46688,36.72455,23.9,-0.00057,-0.00057,-0.00060,0.00015,0.0000,...,2021,6,30,23,50,-0.144802,1.049685,1.210674,0.000000,0.000000
447708,202106302350,978,127.77626,37.95461,23.2,-0.00057,0.00000,0.00030,-0.00075,-0.0003,...,2021,6,30,23,50,-0.365471,0.726721,0.740235,0.000000,0.000000


In [17]:
##### Step 1 - Select data for modeling(training data & validation data
X_train = train.drop(['Band1','Band2','Band3','Band4','Band5','Band6','Band7','Band8','Band9','Band10','Band11','Band12', \
            'Band13','Band14','Band15','Band16','30daysBand3','30daysBand13', \
           'insitu-TED0.05','insitu-TED0.1','insitu-TED0.2','insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED3.0', \
           'insitu-TED5.0','insitu-PA','insitu-PS','Year','Month','Day','Hour','Minute','YearMonthDayHourMinute', \
           'STN','Lon','Lat','isitu-LST','insitu-TA'], axis=1)
X_test = val.drop(['Band1','Band2','Band3','Band4','Band5','Band6','Band7','Band8','Band9','Band10','Band11','Band12', \
            'Band13','Band14','Band15','Band16','30daysBand3','30daysBand13', \
           'insitu-TED0.05','insitu-TED0.1','insitu-TED0.2','insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED3.0', \
           'insitu-TED5.0','insitu-PA','insitu-PS','Year','Month','Day','Hour','Minute','YearMonthDayHourMinute', \
           'STN','Lon','Lat','isitu-LST','insitu-TA'], axis=1)

y_train=train[['isitu-LST', 'insitu-TA']].values
y_test=val[['isitu-LST', 'insitu-TA']].values

X_train

,GK2A-LST,SolarZA,SateZA,ESR,Height,LandType,insitu-HM,insitu-TD,insitu-TG,mean_6_23,mean_34_37,mean_38_39,mean_40_42,mean_43_44
0,-999.00000,162.63634,44.52481,-9.38141,17.53,2,30.3,-17.7,-4.2,0.422121,0.000000,0.000000,0.000000,0.868476
1,-999.00000,162.17793,44.41427,-9.36082,155.48,2,64.1,-14.5,-7.9,-0.391234,0.000000,0.000000,0.000000,1.114533
2,-999.00000,162.27711,44.13104,-9.36664,115.62,2,58.4,-14.7,-7.4,-0.355405,0.000000,0.000000,0.000000,1.315872
3,-999.00000,162.14729,44.12316,-9.36060,30.59,2,52.8,-15.4,-6.5,-0.332510,0.000000,0.000000,0.000000,1.666385
4,-9.38000,162.57755,44.12761,-9.38050,75.82,3,66.0,-14.3,-14.1,-0.188191,-1.022276,-0.499305,0.114355,1.402172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447705,19.20999,120.35766,40.87855,-4.18556,39.64,2,96.9,20.0,19.4,0.000385,1.107682,1.107273,0.000000,0.209894
447706,20.35001,120.38888,41.10588,-4.21799,16.78,2,93.5,18.7,-999.0,0.071010,1.472435,1.488681,0.000000,0.000000
447707,14.64999,118.81387,42.79873,-3.99159,40.35,2,98.3,21.6,21.9,-0.144802,1.049670,1.210674,0.000000,0.000000
447708,-999.00000,117.67587,44.18884,-3.84460,140.37,2,95.9,20.1,19.2,-0.365471,0.726709,0.740235,0.000000,0.000000


In [18]:
X_test

,GK2A-LST,SolarZA,SateZA,ESR,Height,LandType,insitu-HM,insitu-TD,insitu-TG,mean_6_23,mean_34_37,mean_38_39,mean_40_42,mean_43_44
0,-999.00000,132.65269,44.52481,-6.38871,17.53,2,97.3,16.9,17.8,-0.733944,0.000000,0.000000,0.000000,0.508451
1,-999.00000,132.82147,44.16866,-6.41119,95.78,3,98.5,17.5,17.5,-0.696051,-0.914975,0.447435,0.341129,-0.179931
2,-999.00000,132.53152,44.41427,-6.37791,155.48,2,100.0,17.1,17.1,-0.279810,0.000000,0.000000,0.000000,-0.365214
3,13.45001,132.73465,44.13104,-6.40288,115.62,2,99.9,17.7,18.1,-0.195837,0.000000,0.000000,0.000000,-0.290579
4,-999.00000,132.69038,44.12316,-6.39848,30.59,2,99.8,17.7,18.0,-0.370219,0.000000,0.000000,0.000000,0.015434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432625,-9.10999,162.38597,41.92041,-8.86355,15.12,2,82.1,-10.7,-9.7,-0.787369,-1.417396,-1.534663,0.000000,0.000000
432626,-9.66000,162.89288,42.59904,-8.93360,88.62,2,40.1,-17.3,-8.5,-0.672741,-1.492066,-1.894582,0.000000,1.185870
432627,-7.60999,162.98303,40.87855,-8.88743,39.64,2,53.4,-11.7,-4.8,-0.684352,-0.847900,-0.716603,0.000000,1.525860
432628,-999.00000,163.56012,41.10588,-8.94638,16.78,2,36.2,-15.7,-999.0,-2.065335,-1.364155,-0.756098,0.000000,0.000000


In [61]:
# opt = tf.keras.optimizers.Adam(learning_rate=0.1)
opt = tf.keras.optimizers.Adam(lr=0.001)

In [62]:
##### Step 2 - Create training and testing samples
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


##### Step 3 - Specify the structure of a neural network
model2 = Sequential(name="Model-with-Two-Inputs")
model2.add(Input(shape=(len(X_train.columns),), name='Input-Layer'))
model2.add(Dense(len(X_train.columns), activation='relu', name='Hidden-Layer1'))
model2.add(Dense(len(X_train.columns), activation='relu', name='Hidden-Layer2'))
model2.add(Dense(len(X_train.columns), activation='relu', name='Hidden-Layer3'))
model2.add(Dense(2, activation='relu', name='Output-Layer'))


##### Step 4 - Compile the keras model
model2.compile(optimizer=opt,
              loss='mean_squared_error',
              metrics=['mse'],
              loss_weights=None,
              weighted_metrics=None,
              run_eagerly=None, 
              steps_per_execution=None 
             )

model2.summary()

Model: "Model-with-Two-Inputs"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-Layer1 (Dense)       (None, 14)                210       
                                                                 
 Hidden-Layer2 (Dense)       (None, 14)                210       
                                                                 
 Hidden-Layer3 (Dense)       (None, 14)                210       
                                                                 
 Output-Layer (Dense)        (None, 2)                 30        
                                                                 
Total params: 660
Trainable params: 660
Non-trainable params: 0
_________________________________________________________________


In [63]:
cd C:\weather\v3

C:\weather\v3


In [64]:
os.getcwd()

'C:\\weather\\v3'

In [65]:
es = EarlyStopping(monitor='val_mse', verbose=1, patience=10)

In [66]:
##### Step 5 - Fit keras model on the dataset

# Set Directory
save_dir = os.path.join(os.getcwd(),'save')

model_name = 'ffnn_v3.{epoch:02d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)


checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_mse',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint,es]                  

In [67]:
hist = model2.fit(X_train.to_numpy(),       # input data
          y_train,                 # target data
          batch_size=32,    
          epochs=100,         # 늘려야 할 부분     
          validation_data=(X_test, y_test),
          shuffle=True,
          callbacks=callbacks)

Epoch 1/100
256322/256334 [============================>.] - ETA: 0s - loss: nan - mse: nan
Epoch 1: val_mse did not improve from inf
256334/256334 [==============================] - 767s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/100
256334/256334 [==============================] - ETA: 0s - loss: nan - mse: nan
Epoch 2: val_mse did not improve from inf
256334/256334 [==============================] - 615s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/100
256310/256334 [============================>.] - ETA: 0s - loss: nan - mse: nan

KeyboardInterrupt: 

In [ ]:
model2.save('ffnn_v3.h5')

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(hist.history) 

# or save to csv: 
hist_csv_file = 'ffnn_v3_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
hh = pd.read_csv('ffnn_v3_history.csv')
hh

# 시각화

Loss

In [ ]:
def Plot_history(history):
  plt.subplot(211)
  plt.plot(hist.history['loss'], color = 'b', label = 'train')
  plt.plot(hist.history['val_loss'], color = 'g', label = 'test')
  plt.ylabel('Loss')
  plt.legend()

#   plt.subplot(212)
#   plt.plot(hist.history['mse'], color = 'r', label = 'train')
#   plt.plot(hist.history['val_mse'], color = 'black', label = 'test')
#   plt.xlabel('Epoch')
#   plt.ylabel('mse')
#   plt.legend()
#   plt.show()  

In [ ]:
Plot_history(hist)

In [ ]:
scores = model2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test mse:', scores[1])

LST,TA

In [ ]:
# LST,TA 얼마나 잘 맞췄는지 겹쳐 그래프도 그리기

In [ ]:
pred_te = model2.predict(X_test)
pred_te

In [ ]:
lst_pr = []
ta_pr = []
for i in range(len(pred_te)):
    lst_pr.append(pred_te[i][0])
    ta_pr.append(pred_te[i][1])

In [ ]:
lst_re = []
ta_re = []
for i in range(len(y_test)):
    lst_re.append(y_test[i][0])
    ta_re.append(y_test[i][1])

In [ ]:
def value_compare():
  plt.subplot(211)
  plt.plot(lst_pr[500:550], color = 'b', label = 'pred')
  plt.plot(lst_re[500:550], color = 'g', label = 'real')
  plt.ylabel('LST')
  plt.legend()

  plt.subplot(212)
  plt.plot(ta_pr[700:750], color = 'r', label = 'pred')
  plt.plot(ta_re[700:750], color = 'black', label = 'real')
  plt.ylabel('TA')
  plt.legend()
  plt.show()  

In [ ]:
value_compare()

# 예측 값 파일에 저장

In [ ]:
test_ = test.drop(['Band1','Band2','Band3','Band4','Band5','Band6','Band7','Band8','Band9','Band10','Band11','Band12', \
            'Band13','Band14','Band15','Band16','30daysBand3','30daysBand13', \
           'insitu-TED0.05','insitu-TED0.1','insitu-TED0.2','insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED3.0', \
           'insitu-TED5.0','insitu-PA','insitu-PS','Year','Month','Day','Hour','Minute','YearMonthDayHourMinute', \
           'STN','Lon','Lat','isitu-LST','insitu-TA'], axis=1)

In [ ]:
pred = model2.predict(test_)
pred

In [ ]:
submission1 = submission.copy()
submission1

In [ ]:
submission1[['isitu-LST', 'insitu-TA']] = pred
submission1

In [ ]:
cd C:\weather

In [ ]:
submission1.to_csv("ffnn_v3_220224.csv", index = False)    # 접수번호로 저장